<a href="https://colab.research.google.com/github/SARATHKUMARTJ/Item_Demand_Forecasting/blob/main/My_Item_Demand_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Accurate forecasting of demand can help the manufacturers to maintain appropriate stock which results in reduction in loss due to product not being sold and also reduces the opportunity cost (i.e. higher demand but less availability => opportunity lost)**

Data fields

*   **date**  - Date of the sale data. There are no holiday effects or store closures.

*   **store** - Store ID

*   **item**  - Item ID

*  **sales**  - Number of items sold at a particular store on a particular date.











**In this project, the goal is to forecast 3-month sales for 50 different products in 10 different stores when given 5 years of store item sales data.**

In [3]:
# import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn import linear_model
from matplotlib.pyplot import figure


In [ ]:
df= pd.read_csv('/content/IDF.csv')
df

### **DATA CLEANING**

In [ ]:
df.info()

DATA HAS NO MISSING AND ZERO NULL VALUES FOR ALL COLUMNS SO NO NEED OF IMPUTE AND DROP THE DATA

*IN DATE COLUMN  CONVERTED OBJECT DATATYPE INTO DATETIME*

In [6]:
df['date']=pd.to_datetime(df['date'], errors='ignore')

In [ ]:
df['date']

UNDERSTANDING THE DATASET

In [ ]:
df['store'].unique()


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [ ]:
df['store'].nunique()


10

In [ ]:
df.groupby(["store"]).agg({"sales": ["count","sum", "mean", "median", "std", "min", "max"]})

In [ ]:
df.item.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [ ]:
df.item.nunique()

50

In [ ]:
df.groupby(["item"]).agg({"sales": ["count","sum", "mean", "median", "std", "min", "max"]})

*finding outliers*

In [ ]:
df.describe()

In [ ]:
iqr = df.store.quantile(0.75) - df.store.quantile(0.25)
iqr
upper_threshold = df.store.quantile(0.75) + (1.5 * iqr) # q3 + 1.5iqr
lower_threshold = df.store.quantile(0.25) - (1.5 * iqr) # q1 - 1.5iqr
upper_threshold,lower_threshold

(15.5, -4.5)

In [ ]:
#plt.subplot(1, 2, 2)
fig = px.box(df["store"])
fig.show()

AS WE OBSERVED THE GRAPH THEIR  IS NO OUTLIERS IN THE STORE


In [ ]:
iqr = df.item.quantile(0.75) - df.item.quantile(0.25)
iqr
upper_threshold = df.item.quantile(0.75) + (1.5 * iqr) # q3 + 1.5iqr
lower_threshold = df.item.quantile(0.25) - (1.5 * iqr) # q1 - 1.5iqr
upper_threshold,lower_threshold

(75.5, -24.5)

In [ ]:
#plt.subplot(1, 2, 2)
fig = px.box(df["item"])
fig.show()

AS WE OBSERVED THE GRAPH THEIR  IS NO OUTLIERS IN THE ITEM



In [ ]:
iqr = df.sales.quantile(0.75) - df.sales.quantile(0.25)
iqr
upper_threshold = df.sales.quantile(0.75) + (1.5 * iqr) # q3 + 1.5iqr
lower_threshold = df.sales.quantile(0.25) - (1.5 * iqr) # q1 - 1.5iqr
upper_threshold,lower_threshold

(130.0, -30.0)

In [ ]:
#ply.boxplot(df['sales'])
#plt.show()

fig = px.box(df["sales"])
fig.show()

AS WE OBSERVED THE GRAPH THEIR  IS  OUTLIERS IN THE SALES NEED TO CLIP 


In [ ]:
df.sales = df.sales.clip(upper_threshold,lower_threshold)
df.sales
fig = px.box(df["sales"])
fig.show()

AFTER CLIPPING NO OUTLIERS FOUND

In [ ]:
df.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.067088
std,2.872283,14.430878,28.223040
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,130.000000


In [ ]:
plt.subplots(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.distplot(df['sales'])
  
plt.subplot(1, 2, 2)
sns.boxplot(df['sales'])
plt.show()

In [ ]:
df

## **TASK JAR**

>*Finding Linear*



In [ ]:
df.plot(x='item', y='sales', style='o')
plt.title('item vs sales')
plt.xlabel('item')
plt.ylabel('sales')
plt.show()

In [ ]:
df.plot(x='store', y='sales', style='o')
plt.title('store vs sales')
plt.xlabel('store')
plt.ylabel('sales')
plt.show()

In [ ]:
df.plot(x='date', y='sales', style='o')
plt.title('date vs sales')
plt.xlabel('date')
plt.ylabel('sales')
plt.show()

In [ ]:
sales_by_item = df.groupby('item')['sales'].sum().reset_index()
sales_by_item

In [ ]:
#sales_by_item = df.groupby('item')['sales'].sum().reset_index()
fig, ax = plt.subplots(figsize=(20,8))
sns.barplot(sales_by_item.item, sales_by_item.sales, order=sales_by_item.sort_values('sales', ascending = False).item)
    
#ax.set(xlabel = "Item Id", ylabel = "Sum of Sales", title = "Total Sales Per Item")
#plt.xticks(fontsize=12, rotation=90)

In [ ]:
sales_by_store = df.groupby('store')['sales'].sum().reset_index()
sales_by_store 

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
sns.barplot(sales_by_store.store, sales_by_store.sales, order=sales_by_store.sort_values('sales',ascending = False).store)
    
#ax.set(xlabel = "Store Id", ylabel = "Sum of Sales", title = "Total Sales Per Store")
#plt.xticks(fontsize=12, rotation=45)

In [ ]:
figure(figsize=(18, 4), dpi=80)
item_daily = df.groupby(["date","store"],as_index=False).agg({"sales":"sum"})

item_daily['date'] = pd.to_datetime(item_daily.date, format='%Y/%m/%d')
item_1 = item_daily[item_daily['store']==1]
ax_2 = sns.scatterplot(data=item_1,x='date',y='sales')
ax_2.set_ylabel("Sales/store")

In [ ]:
figure(figsize=(18, 4), dpi=80)
item_daily = df.groupby(["date","item"],as_index=False).agg({"sales":"sum"})

item_daily['date'] = pd.to_datetime(item_daily.date, format='%Y/%m/%d')
item_1 = item_daily[item_daily['item']==1]
ax_2 = sns.scatterplot(data=item_1,x='date',y='sales')
ax_2.set_ylabel("Sales/Item_1")

**Feature Engineering**

In [ ]:
# Convert the date column to a datetime object
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# Create new columns for year, month, and day
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [ ]:

from datetime import datetime
import calendar
      
def weekend_or_weekday(year,month,day):
      
    d = datetime(year,month,day)
    if d.weekday()>4:
        return 1
    else:
        return 0
  
df['weekend'] = df.apply(lambda x:weekend_or_weekday(x['year'], x['month'], x['day']), axis=1)
df.head()

In [ ]:
from datetime import date
import holidays
def is_holiday(x):
    india_holidays = holidays.country_holidays('IN')
    if india_holidays.get(x):
      return 1
    else:
      return 0
df['holidays'] = df['date'].apply(is_holiday)
df.head()

In [ ]:

def which_day(year, month, day):
      
    d = datetime(year,month,day)
    return d.weekday()
  
df['weekday'] = df.apply(lambda x: which_day(x['year'],x['month'],x['day']),axis=1)
df.head()

In [ ]:
features = ['store', 'year', 'month',\
            'weekday', 'weekend','holidays' ]
  
plt.subplots(figsize=(20, 10))
for i, col in enumerate(features):
    plt.subplot(2, 3, i + 1)
    df.groupby(col).mean()['sales'].plot.bar()
plt.show()

In [ ]:
item_i = df[df['item']==3]
item_i

In [ ]:
k = item_i.groupby(['date','item'])
item_1 = k.agg(sum)
item_1=item_1.reset_index()
j=[]
for i in range(89, len(item_1)):
    b = item_1['date'][0+i] # 0 is the starting date and 0+i is the end date
    j.append(b)
item = item_1.head(1737) # doubt
item['end']=j    # doubt
date_list = df['date'].to_list()
d =[]
for i in range(1737):
     r = item.loc[i, 'end']
     a = date_list.index(r)
     c =item_1.loc[i:a,'sales'].sum()
     d.append(c)
item['total'] = d
item['date'] = pd.to_datetime(item['date'])
item['year'] = item['date'].dt.year
item['month'] = item['date'].dt.month
item['day'] = item['date'].dt.day

In [ ]:
item

splitting the data into train and test

In [ ]:
x =  item.loc[:,['year','month','day']].values
y = item.loc[:,'total'].values
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.25)

SCALING 

In [ ]:
from sklearn.preprocessing import StandardScaler ## standrard scalig 
scaler = StandardScaler() #initialise to a variable
scaler.fit(x_train) # we are finding the values of mean and sd from the td
x_train = scaler.transform(x_train) # fit (mean, sd) and then transform the training data
x_test= scaler.transform(x_test) # transform the test data 

FITTING LINEAR REGRESSOR

In [ ]:
from sklearn.linear_model import LinearRegression
linear =LinearRegression()
linear.fit(x_train,y_train)
print('score for Linear Regression:',linear.score(x_test,y_test))

score for Linear Regression: 0.3516958579191266


Decision Tree Regressor

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

for depth in [1,2,3,4,5,6,7,8,9,10,20,40,60]:
  dt = DecisionTreeRegressor(max_depth=depth)
  dt.fit(x_train,y_train)
  trainAccuracy = r2_score(y_train,dt.predict(x_train))
  dt = DecisionTreeRegressor(max_depth = depth)
  valAccuracy = cross_val_score(dt, x_train, y_train, cv=10, scoring = make_scorer(r2_score))
  print("Depth:",depth,'Train R2:',trainAccuracy,'Val Score:',np.mean(valAccuracy))
dt = DecisionTreeRegressor(max_depth = int(input('max depth value')))
dt.fit(x_train,y_train)
#print('score for Decision Treeregressor:',dt.score(x_test,y_test))

Depth: 1 Train R2: 0.331759141389831 Val Score: 0.3207263341201472
Depth: 2 Train R2: 0.5673615359427555 Val Score: 0.5325261325081809
Depth: 3 Train R2: 0.7668734007645478 Val Score: 0.7497929278153517
Depth: 4 Train R2: 0.8739516808974007 Val Score: 0.8721695101922512
Depth: 5 Train R2: 0.9374246871382257 Val Score: 0.9366783769078886
Depth: 6 Train R2: 0.9665410845527251 Val Score: 0.963674451534802
Depth: 7 Train R2: 0.9861400814278374 Val Score: 0.9825633532502355
Depth: 8 Train R2: 0.9941695432363306 Val Score: 0.9909371432005691
Depth: 9 Train R2: 0.9975836980153664 Val Score: 0.9950186258324182
Depth: 10 Train R2: 0.9993371418736229 Val Score: 0.9967452908340733
Depth: 20 Train R2: 1.0 Val Score: 0.9976276547939857
Depth: 40 Train R2: 1.0 Val Score: 0.9976276547939857
Depth: 60 Train R2: 1.0 Val Score: 0.9976276547939857
max depth value7


DecisionTreeRegressor(max_depth=7)

KNeighborsRegressor

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
for i in [1,2,3,4,5,6,7,8,9,10,20,50]:
  knn = KNeighborsRegressor(i)
  knn.fit(x_train,y_train)
  print('k value:',i ,'train score:',knn.score(x_train,y_train),'cv score:',np.mean(cross_val_score(knn, x_train, y_train, cv=10, scoring = make_scorer(r2_score))))
knn =KNeighborsRegressor(int(input('enter k values:')))
knn.fit(x_train,y_train)
#print('score for knn regression :',knn.score(x_test,y_test))

k value: 1 train score: 1.0 cv score: 0.9947139489485716
k value: 2 train score: 0.9992117826845966 cv score: 0.9906568886740837
k value: 3 train score: 0.9976249761448945 cv score: 0.9846643104253611
k value: 4 train score: 0.9944892072983942 cv score: 0.9811921641601069
k value: 5 train score: 0.9912125511000343 cv score: 0.9777273084802196
k value: 6 train score: 0.9878035887455066 cv score: 0.9760886595655908
k value: 7 train score: 0.9852831213627008 cv score: 0.9757293378408125
k value: 8 train score: 0.9838483644982986 cv score: 0.9742088991858985
k value: 9 train score: 0.9834030524256628 cv score: 0.9718755996925065
k value: 10 train score: 0.9818160113084127 cv score: 0.9701070322978147
k value: 20 train score: 0.9601711540083331 cv score: 0.9493580482060391
k value: 50 train score: 0.9344672809647872 cv score: 0.9219162084683612
enter k values:10


KNeighborsRegressor(n_neighbors=10)

!pip install -U scikit-learn

In [ ]:
!pip install -U scikit-learn

In [ ]:
from xgboost import XGBRegressor
import xgboost as xgb

for lr in [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.2,0.5,0.7,1]:
  model = xgb.XGBRegressor(learning_rate = lr,n_estimators =100,verbosity =0)#initialise the model
  model.fit(x_train,y_train)
  model.score(x_test,y_test)
  print('Learning rate:',lr,"Train score",model.score(x_train,y_train),'Cross-Val score:',np.mean(cross_val_score(model,x_train,y_train,cv=10)))
model = xgb.XGBRegressor(learning_rate = float(input('LR value')),n_estimators =100) 
model.fit(x_train,y_train)

#print('score for the XGBRegressor:',model.score(x_test,y_test))

Learning rate: 0.01 Train score -3.3354036462639804 Cross-Val score: -3.4309738747735126
Learning rate: 0.02 Train score 0.3733168956637243 Cross-Val score: 0.3565095013945786
Learning rate: 0.03 Train score 0.8930563020925969 Cross-Val score: 0.8892427335609773
Learning rate: 0.04 Train score 0.9707062937137009 Cross-Val score: 0.9685348777056836
Learning rate: 0.05 Train score 0.9858735467623858 Cross-Val score: 0.9840552614536613
Learning rate: 0.06 Train score 0.9911608948619217 Cross-Val score: 0.9897937538219244
Learning rate: 0.07 Train score 0.9943511463862982 Cross-Val score: 0.9932252247448463
Learning rate: 0.08 Train score 0.9962062601601827 Cross-Val score: 0.995124459796893
Learning rate: 0.09 Train score 0.9970431921870445 Cross-Val score: 0.9959919473261782
Learning rate: 0.1 Train score 0.9975691246954981 Cross-Val score: 0.9965575463477279
Learning rate: 0.11 Train score 0.99756641319123 Cross-Val score: 0.9966483693997692
Learning rate: 0.12 Train score 0.99808932984

XGBRegressor(learning_rate=0.7)

In [ ]:
print('Score for the Linear Regressor       :',linear.score(x_test,y_test))    
print('Score for the Decision TreeRegressor :',dt.score(x_test,y_test))    
print('Score for the KNN Regressor          :',knn.score(x_test,y_test)) 
print('Score for the XGBRegressor           :',model.score(x_test,y_test))

Score for the Linear Regressor       : 0.3516958579191266
Score for the Decision TreeRegressor : 0.9818720924853016
Score for the KNN Regressor          : 0.9691072501054501
Score for the XGBRegressor           : 0.9993039950486162


In [ ]:
linear_pred = linear.predict(x_test)
dt_pred= dt.predict(x_test)
knn_pred=knn.predict(x_test)
xgb_pred=model.predict(x_test)

In [ ]:
pd.DataFrame({"Actual":y_test, "linear_pred":linear_pred,"dt_pred":dt_pred, "knn_pred":knn_pred,"xgb_pred":xgb_pred })

,Actual,linear_pred,dt_pred,knn_pred,xgb_pred
0,44686.0,38974.855064,43131.405594,44376.5,44951.828125
1,27452.0,29427.038915,27275.000000,27094.4,27593.417969
2,36520.0,35004.725838,36365.818182,36293.7,36501.437500
3,31752.0,32582.467732,31821.700000,32223.9,31904.962891
4,35230.0,35181.575292,36673.315789,35935.3,35092.058594
...,...,...,...,...,...
430,36936.0,35670.069184,37092.826087,37587.5,37075.835938
431,24196.0,26658.362941,24305.000000,24256.6,23899.521484
432,40099.0,34248.176994,39263.171875,39569.3,40182.277344
433,37490.0,37485.135508,38278.000000,37653.4,37504.265625
